In [82]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer, GenerationConfig
from peft import PeftModel, PeftConfig
import torch
import copy


torch_dtype = torch.bfloat16
base_model_dir = "/data/model/Qwen2.5-Math-7B"
lora_model_dir = "/data/ER-GRPO/Qwen2.5-Math-7B-data/GRPO/checkpoint-100"
device_map = "cpu"
print("Loading base model and tokenizer...")
print("Loading base model and tokenizer...")
model_base = AutoModelForCausalLM.from_pretrained(
    base_model_dir,
    load_in_8bit=False,
    torch_dtype=torch_dtype,
    device_map=device_map,
   

)
tokenizer = AutoTokenizer.from_pretrained(base_model_dir)
model_base_original = copy.deepcopy(model_base)

print("Loading LoRA configuration...")
peft_config = PeftConfig.from_pretrained(lora_model_dir)

print("Loading LoRA model and applying weights...")
model_lora = PeftModel.from_pretrained(
    model_base,
    lora_model_dir,
    torch_dtype=torch_dtype,
    device_map=device_map,
    is_trainable=True,
    inference_mode=False # 明确禁用推理模式，保证LoRA激活
)




Loading base model and tokenizer...
Loading base model and tokenizer...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 71.47it/s]


Loading LoRA configuration...
Loading LoRA model and applying weights...


In [85]:
# print("Merging LoRA weights into base model...")
model_merged = model_lora.merge_and_unload()

In [61]:
import os
print(os.listdir(lora_model_dir))

['tokenizer.json', 'merges.txt', 'adapter_config.json', 'latest', 'chat_template.jinja', 'added_tokens.json', 'vocab.json', 'tokenizer_config.json', 'rng_state_1.pth', 'scheduler.pt', 'zero_to_fp32.py', 'README.md', 'rng_state_0.pth', 'global_step100', 'adapter_model.safetensors', 'trainer_state.json', 'training_args.bin', 'special_tokens_map.json']


In [62]:
print("LoRA modules:", model_lora.peft_config)

LoRA modules: {'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/data/model/Qwen2.5-Math-7B', revision=None, inference_mode=False, r=32, target_modules={'down_proj', 'up_proj', 'v_proj', 'o_proj', 'q_proj', 'gate_proj', 'k_proj'}, exclude_modules=None, lora_alpha=64, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}


In [70]:
import torch
from safetensors.torch import load_file


print("Loaded state dict keys:", state_dict.keys())




Loaded state dict keys: dict_keys(['base_model.model.model.layers.0.mlp.down_proj.lora_A.weight', 'base_model.model.model.layers.0.mlp.down_proj.lora_B.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.weight', 'base_model.model.model.layers.1.mlp.down_proj.lora_A.weight', 

In [46]:
import torch
def compare_model_weights(model1, model2):
    for name1, param1 in model1.named_parameters():
        if name1 in model2.state_dict():
            param2 = model2.state_dict()[name1]
            if not torch.allclose(param1, param2, rtol=1e-03, atol=1e-03):
                print(f"Layer '{name1}': Weights are DIFFERENT.")
                return True
        else:
            print(f"Layer '{name1}' not found in the second model.")
            return True
    return False


In [47]:
compare_model_weights(model_base_original, model_merged)  # Should be True

False

In [58]:
params2 = list(model_lora.named_parameters())

In [86]:

# 获取参数
params1 = list(model_base_original.parameters())
params2 = list(model_merged.parameters())

# 比较参数
for p1, p2 in zip(params1, params2):
    if torch.allclose(p1.data, p2.data):
        print("Parameters are equal.")
    else:
        print("Parameters are not equal.")



Parameters are equal.
Parameters are not equal.
Parameters are equal.
Parameters are not equal.
Parameters are equal.
Parameters are not equal.
Parameters are equal.
Parameters are not equal.
Parameters are not equal.
Parameters are not equal.
Parameters are not equal.
Parameters are equal.
Parameters are equal.
Parameters are not equal.
Parameters are equal.
Parameters are not equal.
Parameters are equal.
Parameters are not equal.
Parameters are equal.
Parameters are not equal.
Parameters are not equal.
Parameters are not equal.
Parameters are not equal.
Parameters are equal.
Parameters are equal.
Parameters are not equal.
Parameters are equal.
Parameters are not equal.
Parameters are equal.
Parameters are not equal.
Parameters are equal.
Parameters are not equal.
Parameters are not equal.
Parameters are not equal.
Parameters are not equal.
Parameters are equal.
Parameters are equal.
Parameters are not equal.
Parameters are equal.
Parameters are not equal.
Parameters are equal.
Parame

In [ ]:
params1 = {name: param.clone().detach() for name, param in model_base_original.named_parameters()}
params2 = {name: param.clone().detach() for name, param in model_merged.named_parameters()}
are_same = True
for name in params1.keys():
    if name not in params2:
        print(f"Parameter {name} not found in model2")
        are_same = False
        break
    if not torch.allclose(params1[name], params2[name], atol=1e-6):
        print(f"Parameter {name} differs")
        are_same = False
        break

if are_same:
    print("All parameters are identical")
else:
    print("Models have different parameters")

Parameter model.layers.0.self_attn.q_proj.weight differs
Models have different parameters


: 